##Import and Download

In [ ]:
import os
from google.colab import drive

# Mount the Google drive storage
drive.mount('/content/drive')

# Move to the directory with the notebooks and the data set
# (note, you may need to change this path according to your directory structure)
os.chdir('/content/drive/MyDrive/')

# Check the currect working directory
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive'

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 119.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 805.4 kB/s eta 0:00:00


##Read the Dataset

In [ ]:
import json
def load_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data['data']

train_raw = load_data('train-v2.0.json')

valid_raw = load_data('dev-v2.0.json')

In [ ]:
want_unanswerable = True # True: unanswerable questions are considered (Squad 2), False: thery are not cosidered (Squad 1)

In [ ]:
def extract_train_dataset(data, unanswerable):
    contexts = []
    questions = []
    answers = []
    
    for article in data:
        for paragraph in article['paragraphs']: 
            for qas in paragraph['qas']:
                if qas['is_impossible'] == False:
                    contexts.append(paragraph['context']) #duplicate contexts, s.t. I have one context-one question  
                    questions.append(qas['question'])                
                    answers.append(qas['answers'][0]['text'])
                elif unanswerable == True:
                    contexts.append(paragraph['context']) #duplicate contexts, s.t. I have one context-one question  
                    questions.append(qas['question'])
                    answers.append(" ")

    return contexts, questions, answers

train_dataset_contexts, train_dataset_questions, train_dataset_answers = extract_train_dataset(train_raw, want_unanswerable)
#valid_dataset_contexts, valid_dataset_questions, valid_dataset_answers = extract_contexts_questions(valid_raw)

In [ ]:
print(train_dataset_contexts[0])
print(train_dataset_questions[0])
print(train_dataset_answers[0])

Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
When did Beyonce start becoming popular?
in the late 1990s


In [ ]:
def extract_valid_dataset(data, unanswerable):
    dataset = []

    for article in data:
        for paragraph in article['paragraphs']:
            for qas in paragraph['qas']:
                if qas['is_impossible'] == False: 
                    list_answers = []
                    list_answers_start = []
                    for a in qas['answers']:
                        list_answers.append(a['text'])
                        list_answers_start.append(a['answer_start'])
                    dataset.append({'id': qas['id'],
                                    'context': paragraph['context'],
                                   'question': qas['question'],
                                   'answers': {'text': list_answers, 'answer_start': list_answers_start},
                    })
                elif unanswerable == True:
                    dataset.append({'id': qas['id'],
                                    'context': paragraph['context'],
                                   'question': qas['question'],
                                   'answers': {'text' : ['unanswerable'], 'answer_start': [0]}
                    })

    return dataset

valid_dataset = extract_valid_dataset(valid_raw, want_unanswerable)

In [ ]:
valid_dataset[0]

{'id': '56ddde6b9a695914005b9628',
 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'question': 'In what country is Normandy located?',
 'answers': {'text': ['France', 'France', 'France', 'France'],
  'answer_start': [159, 159, 159, 159]}}

##Preprocessing

In [ ]:
max_length = 382
stride = 128
num_train_epochs = 3
learning_rate = 2e-5

n_best = 20
max_answer_length = 50

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from torch.utils.data import Dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer#, AdamW

from tqdm import tqdm

# Load tokenizer and model
model_name = "gpt2"
#model_name = "distilgpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(model_name)

In [ ]:
class ContextQuestionDataset(Dataset):
    def __init__(self, contexts, questions, answers, tokenizer):
        self.contexts = contexts
        self.questions = questions
        self.answers = answers
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.contexts)

    def __getitem__(self, index):
        context = self.contexts[index]
        question = self.questions[index]
        answer = self.answers[index]

        # Combine context and question
        input_text = f"{context} {question}" 

        # Tokenize the input text and answer
        encoding = self.tokenizer(
            input_text,
            answer,
            add_special_tokens=True,
            max_length=max_length,
            truncation="only_first",
            padding="max_length",
            return_tensors="pt",
            #truncation=True,
            return_overflowing_tokens=True,
            stride=stride
        )

        input_ids = encoding["input_ids"].squeeze()
        attention_mask = encoding["attention_mask"].squeeze()
        labels = encoding["input_ids"].squeeze()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

# Create an instance of the dataset
train_dataset = ContextQuestionDataset(train_dataset_contexts, train_dataset_questions, train_dataset_answers, tokenizer)
#valid_dataset = ContextQuestionDataset(valid_dataset_contexts, valid_dataset_questions, valid_dataset_answers, tokenizer)

##Training

In [ ]:
from transformers import Trainer, TrainingArguments, default_data_collator

training_args = TrainingArguments(
    model_name+'_c-q_a_with_unanswerable',
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=learning_rate,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    #fp16=True #to be used when Cuda is active
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Provide your train dataset here
    eval_dataset=valid_dataset,  # Provide your validation dataset here
    data_collator=default_data_collator,  # Optional: if you need custom data collation
    tokenizer=tokenizer,  # Optional: if you need a tokenizer for text generation during evaluation
)
#trainer.train() #uncomment to train. Note that GPU is needed. If want to use CPU, comment "fp16=True" in TrainingArguments

##Evaluation

In [ ]:
# import model from checkpoint
import torch

model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/gpt2_c-q_a_with_unanswerable/checkpoint-48870')
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
context = "Natural Language Processing (NLP) concerns the computational analysis, interpretation, and production of natural language in either written or spoken form. It is an interdisciplinary research field, interesting from both theoretical and practical perspectives. Decades of research have resulted in a vast collection of symbolic, stochastic, and deep-learning based models. Such models have enable the development of applications in a vast array of fields, such as human-machine interaction and chatbots, search and question answering, translation and multilingual systems, multimodal and captioning systems, speech analysis, voice interaction and personal assistants, sentiment analysis, etc, etc. This course will provide an introduction to the important problems, models and applications in NLP. The history of NLP involves many successes and many failures, demonstrating the complexity of the topic. Initially popular symbolic models turned out to be unable to capture the intrinsic complexity of natural language. Statistical techniques such as vector-space representations and linear classifiers (e.g. Support Vector Machines) enabled important applications such as web search spam detection. Word embedding techniques then became popular and improved performance on all aspects of NLP: from morphology to semantics and dialogue. More recently sequence-to-sequence modeling with deep learning techniques have greatly improved performance on hard NLP problems such machine translation and dialog generation."
#question = "Who is the subject of the context?"
#question = "Which statistical techniques are used?"
question = "What is NLP?"
#question = "What is Natural Language Processing about?"
#input_text = f"Context: {context} Question: {question}" 
input_text = f"{context} {question}" 

input_ids = tokenizer.encode(input_text, add_special_tokens=True, return_tensors="pt")
attention_mask = torch.ones_like(input_ids)  # Set attention mask to 1 for all tokens
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=1000, do_sample=True, num_return_sequences=1, pad_token_id=50256)
answer = tokenizer.decode(output[0], skip_special_tokens=True)
print(answer[len(input_text) :])

the computational analysis, interpretation, and production of natural language


###Metrics

Metrics **without** sampling

In [ ]:
import evaluate
import collections
import numpy as np

predicted_answers = []
real_answers = []
for valid_group in valid_dataset:
    # set current id (this is necessary for the function that will compute the metrics)
    curr_id = valid_group['id']
    # put context and question together
    input_text = f"{valid_group['context']} {valid_group['question']}" 
    # prepare the input
    input_ids = tokenizer.encode(input_text, add_special_tokens=True, return_tensors="pt")
    attention_mask = torch.ones_like(input_ids)  # Set attention mask to 1 for all tokens
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    # get the generated answer
    output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=1000, do_sample=False, num_return_sequences=1)
    output_string = tokenizer.decode(output[0], skip_special_tokens=True)[len(input_text):]
    # save result
    if output_string == " ":
        output_string = "unanswerable"
    predicted_answers.append({'id' : curr_id, 'prediction_text' : output_string})
    real_answers.append({'id' : curr_id, 'answers' : valid_group['answers']})

metric = evaluate.load("squad")
metric.compute(predictions=predicted_answers, references=real_answers)

Output streaming troncato alle ultime 5000 righe.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_t

{'exact_match': 50.33268761054494, 'f1': 52.945564500293216}

Metrics **with** sampling

In [ ]:
import evaluate
import collections
import numpy as np

predicted_answers = []
real_answers = []
for valid_group in valid_dataset:
    # set current id (this is necessary for the function that will compute the metrics)
    curr_id = valid_group['id']
    # put context and question together
    input_text = valid_group['question'] 
    # prepare the input
    input_ids = tokenizer.encode(input_text, add_special_tokens=True, return_tensors="pt")
    attention_mask = torch.ones_like(input_ids)  # Set attention mask to 1 for all tokens
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    # get the generated answer
    output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=1000, do_sample=True, num_return_sequences=1)
    output_string = tokenizer.decode(output[0], skip_special_tokens=True)[len(input_text):]
    # save result
    if output_string == " ":
        output_string = "unanswerable"
    predicted_answers.append({'id' : curr_id, 'prediction_text' : output_string})
    real_answers.append({'id' : curr_id, 'answers' : valid_group['answers']})

metric = evaluate.load("squad")
metric.compute(predictions=predicted_answers, references=real_answers)

Output streaming troncato alle ultime 5000 righe.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_t

{'exact_match': 32.805525141076394, 'f1': 33.720019502870045}

In [ ]:
print("Architure: ")
print(model)

Architure: 
GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters:", total_params)

Total number of parameters: 124439808
